In [1]:
from pickle import dump
from pickle import load
import os
import numpy as np

import string
from keras.models import Model

from keras.applications.vgg16 import VGG16
from keras.preprocessing.image import img_to_array
from keras.preprocessing.image import load_img
from keras.applications.vgg16 import preprocess_input

from keras.preprocessing.text import Tokenizer

from keras.utils.np_utils import to_categorical

from keras.preprocessing.sequence import pad_sequences

from keras.callbacks import ModelCheckpoint


from keras import Sequential
from keras.layers import Embedding, LSTM, Dense, Dropout, Input

from keras.layers.merge import add

from keras.utils import plot_model

from nltk.translate.bleu_score import corpus_bleu



#jupyter notebook --NotebookApp.iopub_data_rate_limit=1.0e10

Using TensorFlow backend.


In [2]:
model = VGG16()
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

In [3]:
model.layers.pop()
model = Model(inputs=model.inputs, outputs=model.layers[-1].output)

In [4]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

In [5]:
#'description' in link mlmastery = 'mapping' in this code which are dictionaries of {id:[desc1,desc2,desc3]}
def inputimage(directory):
    features = dict()
    processed_imagescount = 0
    for filename in os.listdir(directory):
        comp_filename = directory+"/"+filename
        
        print(comp_filename)
        image=load_img(comp_filename,target_size=(224,224))
        
        image=img_to_array(image)
        
        print(image.shape,image.size)
        image=image.reshape((1,image.shape[0],image.shape[1],image.shape[2]))
        
        print(image.shape,image.size)
        image=preprocess_input(image)
        
        print(image.shape,image.size)
#
        filename = filename.split('.')[0]
    
        prediction = model.predict(image,verbose=0)
        
        processed_imagescount+=1
        
        print('processed_imagescount = ',processed_imagescount)
    
        features[filename] = prediction
    
        #print(type(prediction))
        #print(prediction.shape)
        #print(prediction.argmax(axis=1))
        
    return features

In [6]:
h=0
directory = "Flicker8k_Dataset"
for filename in os.listdir(directory):
    h+=1 
print(h)

8091


In [7]:
ls

BS.ipynb             Untitled2.ipynb      model.png
Flicker8k_Dataset/   cleanedcaptions.txt  sele.ipynb
Flickr8k_text/       fakeimg/             submission.csv
Newint.ipynb         features.pkl         tokenizer.pkl
Untitled.ipynb       gncp.ipynb
Untitled1.ipynb      gncp2.ipynb


In [8]:
'''
directory = "Flicker8k_Dataset"#"Flicker8k_Dataset"
features = inputimage(directory)
#saving features
dump(features,open("features.pkl","wb"))
'''

'\ndirectory = "Flicker8k_Dataset"#"Flicker8k_Dataset"\nfeatures = inputimage(directory)\n#saving features\ndump(features,open("features.pkl","wb"))\n'

In [9]:
#run all above
features = load(open('features.pkl','rb'))

In [10]:
len(features)

8091

In [ ]:
#loading data from Flickr8k.token.txt
def load_data(filename):
    file = open(filename,'r')
    text = file.read()
    file.close()
    return text

filename = "Flickr8k_text/Flickr8k.token.txt"
data = load_data(filename)
data

In [ ]:
#mapping image and all its captions

def mapping(data):
    img_data_map = dict()
    t=0
    for line in data.split("\n")[:-1]:#iterate till 2nd last element
        t=t+1
        #print(line)
        token = line.split()
        #print(token)
        img_id = token[0]
        img_desc = token[1:] 
        #print(img_id," -> ",img_desc)
        img_desc = ' '.join(img_desc)

        img_id_ = img_id.split('.')[0]
        #print(img_id_)
        print(img_id_,img_desc)        
        if img_id_ not in img_data_map:
            img_data_map[img_id_]=list()
        img_data_map[img_id_].append(img_desc)
        
    return img_data_map,t
t=0
mapping,t = mapping(data)

In [ ]:
#print(type(imgid_data_map))
for i in mapping:
    print(i,"  ",mapping[i])

In [ ]:
#data cleaning
def clean_descriptions(mapping):
    for i in mapping:
        #lowercase
        for j in range(0,len(mapping[i])):
            #print(type(mapping[i]))
            #print(type(mapping[i][j]))
            print(mapping[i][j])
            mapping[i][j]=mapping[i][j].lower();
            mapping[i][j].translate(str.maketrans('', '', string.punctuation))
            mapping[i][j]=' '.join([w for w in mapping[i][j].split() if len(w)>1])
            mapping[i][j]=' '.join([w for w in mapping[i][j].split() if w.isalpha()])
    '''    
    '''
mapping = clean_descriptions(mapping)

In [ ]:
mapping

In [16]:
b = 'sdf  , sd 100 df'
#' '.join(w for w in mapping[i][j] if w.isalpha())
' '.join([w for w in b.split() if w.isalpha()])

'sdf sd df'

In [17]:

def build_vocab(mapping):
    vocab=set()
    for i in mapping:
        for j in range(0,len(mapping[i])):
            words=mapping[i][j].split()
            for k in words:
                vocab.add(k)
    return vocab

In [18]:
vocab=build_vocab(mapping)
#vocab

In [ ]:
def save_mapping(mapping,filename):
    element=[]
    tot=0
    for img,descs in mapping.items():
        for desc in descs:
            tot=tot+1
            element.append(img+' '+desc)
    data='\n'.join(element)
    #print(element)
    file = open(filename,'w')
    file.write(data)
    file.close()
    return tot
    

In [ ]:
total_descs=save_mapping(mapping,'cleanedcaptions.txt')

In [ ]:
len(mapping)

8092

In [ ]:
total_descs

40460

In [ ]:
len(vocab)

8357

In [ ]:
#get the img without extension
def filename(filename):
    files=set()
    file = open(filename,'r')
    text = file.read();
    file.close()
    
    lst = list()
    lst=text.split('\n')
    for i in range(0,len(lst)):
        #print(lst[i],'\n')
        lst[i] = lst[i].split('.')[0]
        print(lst[i],'\n')
        
    return lst
    #return files

In [ ]:
train_id=filename('Flickr8k_text/Flickr_8k.trainImages.txt')
len(train_id)

In [ ]:
'Flickr8k_text/Flickr_8k.trainImages.txt'

'Flickr8k_text/Flickr_8k.trainImages.txt'

In [ ]:
'''
#return dict of a 
{
    image_id : [---,---,---]
}
#for those img_id present in training set. Same can be done for development set ..

'''
def load_details(filename):
    file = open(filename,'r')
    text = file.read()
    file.close()
    #print(text)
    return text

#descriptions.txt = cleanedcaptions.txt


# We have the train.txt file which contains the imageid of train images. 
#We make a dictionary named "specific_clean_desc" and we take each image id  from the description.txt which we
#saved previously which contains "id : caption" pair. id can be repeated bcz a single id has may captions.
#We loop through each of these pair and if any one of these is present in the train_id then we add 'startseq' and 'endseq'
#in the caption(from description.txt) and make a {id1:[cap1,cap2,cap3], id2:[cap1,cap2,cap3]} and  return this dictionary.

def load_desc(cleanedcaptions,dataset):
    specific_clean_desc = dict()
    id_desc_pair = load_details(cleanedcaptions)
    for line in id_desc_pair.split('\n'):
        #print(line)
        imgid , desc = line.split()[0] , line.split()[1:]
#        print(type(desc))
        if imgid in dataset:
#            print("k")
            newdesc = 'startseq ' + ' '.join(desc) + ' endseq'
#            print(newdesc)
            if imgid not in specific_clean_desc:
#                print("not")
                specific_clean_desc[imgid] = list()
            specific_clean_desc[imgid].append(newdesc)
            #print(specific_clean_desc[imgid])
    return specific_clean_desc
train_descs = load_desc("cleanedcaptions.txt",train_id)


In [ ]:
print(len(train_descs))
train_descs

In [ ]:
#checking that all of the train images are present in the description.txt as ids.
x=0
for key in train_descs:
    if key in train_id:
        #print('uea')
        x=x+1
print(x)

6000


In [ ]:
print(len(train_id))
#reloading the features from feature.pkl
#We saved the 4096 dimensional vector for each image in the the entire images dataset(contained the images).
#features relevant to the train dataset is extracted
from pickle import load
def load_features(pklfile, train_imgid):
    
    features_vector = load(open(pklfile,'rb'))#load(filehandler) is used to load a given dumped file
    #print(len(all_features))
    #print(type(all_features))
    train_features = dict()
    
    for imgid in train_imgid:
        #print('>>',imgid)
        #print(len(all_features[imgid]))
        #print(train_features[imgid]) #
        train_features.update({ imgid : features_vector[imgid] })
        print(features_vector[imgid])
        #adding features of train image ids to a dictionary

    return train_features
                    

train_features = load_features('features.pkl',train_id)

In [ ]:
print(train_features)

In [ ]:
#given a dictionary of {id1:[cap1,cap2,cap3], id2:[cap1,cap2,cap3]} (this we made for the train set where ids 
#are train set images id and different captions associated with an id), create a list of all the caps. ie [cap1,cap2,cap3]
#refer for https://github.com/keras-team/keras/issues/7551
def to_list(dictof_train_idcap):

    text_descs = list()
    
    for ids in dictof_train_idcap:
        for caption in dictof_train_idcap[ids]:
            text_descs.append(caption)
    return text_descs

def create_tokenizer(dictof_train_idcap):
    lines = to_list(dictof_train_idcap)
#    print(type(lines))
#    print(lines)
 
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(lines)
    return(tokenizer)
 
        
tokenizer = create_tokenizer(train_descs)
#print(tokenizer)

In [ ]:
print(tokenizer)

In [ ]:
tokenizer.word_index

In [ ]:
#total vocab size is 8357 and train vocab size is 7266 which means that training set caption has lesser no. of words
#and we may see unseen words in test, valid sets
vocab_size = len(vocab)
print(vocab_size)

8357


In [ ]:
train_vocab_size  = len(tokenizer.word_index) + 1
#to check why 1 is added got to https://machinelearningmastery.com/use-word-embedding-layers-deep-learning-keras/ check under 
#2. Keras Embedding Layer
train_vocab_size

7266

In [ ]:
'''
X1,		X2 (text sequence), 						y (word)
photo	startseq, 									little
photo	startseq, little,							girl
photo	startseq, little, girl, 					running
photo	startseq, little, girl, running, 			in
photo	startseq, little, girl, running, in, 		field
photo	startseq, little, girl, running, in, field, endseq
'''

'\nX1,\t\tX2 (text sequence), \t\t\t\t\t\ty (word)\nphoto\tstartseq, \t\t\t\t\t\t\t\t\tlittle\nphoto\tstartseq, little,\t\t\t\t\t\t\tgirl\nphoto\tstartseq, little, girl, \t\t\t\t\trunning\nphoto\tstartseq, little, girl, running, \t\t\tin\nphoto\tstartseq, little, girl, running, in, \t\tfield\nphoto\tstartseq, little, girl, running, in, field, endseq\n'

In [ ]:
def maxlensentence(train):
    mx = 0
    for id in train:
        for sentence in train[id]:
            #print(type(sentence))
            #print(type(sentence.split()))
            mx= max(mx,len(sentence.split()))
    return mx
maxlen = maxlensentence(train_descs)
print(maxlen)

33


In [ ]:
#it iterates over all the train_descs(which contain {id:[desc1,desc2,desc3]} of only train dataset)
# and for each for each id we pick each of its descriptions and divide it into
#"multiple input sequences" and the predicted sequence.
#eg - [1,2,3,4,5]
#[1],[2]
#[1,2], [3]
#[1,2,3], [4]
##[1,2,3,4], [5]
# we need:
#the tokenizer to convert each word to a number
#feature_vector(4096) #train_features is a dictionary and not a list. {id:[4096 element],id:[4096 element]}
#dictionary containing the train {id:[desc1,desc2,desc3]} which is used to loop through
#maxlen to pad the smaller sentences
#mapping contain mapping of all ids and description in {id:[desc1,desc2,desc3]} format.
def create_sequences(tokenizer,train_descs,train_features,maxlen):
    X1, X2, y = list(), list(), list()
    for key, descs in train_descs.items():
        for desc in descs:
            
            #print(type(desc))
            seqofnumber = tokenizer.texts_to_sequences([desc])[0]
            #print('seqofnumber = ',seqofnumber)

            
            for i in range(1,len(seqofnumber)):##edit : len(seqofnumber)-1
                inseq, outseq = seqofnumber[0:i], seqofnumber[i]
                print('\t',inseq,outseq)
                inseq = pad_sequences([inseq],maxlen = maxlen)[0]
                outseq = to_categorical([outseq],num_classes = train_vocab_size)[0]
                print('outseq',outseq.shape)
                print(key)
                X1.append(train_features[key][0])#train_features is a dictionary and not a list.
                #train_features = {'id' : [], 'id': []}
                X2.append(inseq)
                y.append(outseq)
                #print("\ny :",y.shape)
                
    return np.array(X1), np.array(X2), np.array(y)
X1train, X2train, ytrain = create_sequences(tokenizer,train_descs,train_features,maxlen)

In [ ]:
to_categorical([3],num_classes = 5)[0]

array([0., 0., 0., 1., 0.], dtype=float32)

In [ ]:
ytrain.shape

(305617, 7266)

In [ ]:
t  = Tokenizer()
fit_text = "The earth is an awesome place live"
t.fit_on_texts(fit_text)
test_text = "The earth is an great place live"
sequences = t.texts_to_sequences(test_text)

print("sequences : ",sequences,'\n')

print("word_index : ",t.word_index)
#[] specifies : 1. space b/w the words in the test_text    2. letters that have not occured in fit_text

sequences :  [[3], [4], [1], [], [1], [2], [8], [3], [4], [], [5], [6], [], [2], [9], [], [], [8], [1], [2], [3], [], [13], [7], [2], [14], [1], [], [7], [5], [15], [1]] 

word_index :  {'e': 1, 'a': 2, 't': 3, 'h': 4, 'i': 5, 's': 6, 'l': 7, 'r': 8, 'n': 9, 'w': 10, 'o': 11, 'm': 12, 'p': 13, 'c': 14, 'v': 15}


In [ ]:

t  = Tokenizer()
fit_text = ["The earth is an awesome place live"]
t.fit_on_texts(fit_text)

#fit_on_texts fits on sentences when list of sentences is passed to fit_on_texts() function. 
#ie - fit_on_texts( [ sent1, sent2, sent3,....sentN ] )

#Similarly, list of sentences/single sentence in a list must be passed into texts_to_sequences.
test_text1 = "The earth is an great place live"
test_text2 = "The is my program"
sequences = t.texts_to_sequences([test_text1, test_text2])

print('sequences : ',sequences,'\n')

print('word_index : ',t.word_index)
#texts_to_sequences() returns list of list. ie - [ [] ]

sequences :  [[1, 2, 3, 4, 6, 7], [1, 3]] 

word_index :  {'the': 1, 'earth': 2, 'is': 3, 'an': 4, 'awesome': 5, 'place': 6, 'live': 7}


In [ ]:
l = [0, 2, 1, 2, 0]
print('Ex1:',to_categorical(l),'\n\nEx2: ')
print(to_categorical(l,num_classes = 5))

Ex1: [[1. 0. 0.]
 [0. 0. 1.]
 [0. 1. 0.]
 [0. 0. 1.]
 [1. 0. 0.]] 

Ex2: 
[[1. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0.]
 [0. 1. 0. 0. 0.]
 [0. 0. 1. 0. 0.]
 [1. 0. 0. 0. 0.]]


In [ ]:

#Embedding(7, 2, input_length=5)

#The first argument (7) is the number of distinct words in the training set. 
#The second argument (2) indicates the size of the embedding vectors.
#The input_length argumet, of course, determines the size of each input sequence.


In [ ]:

test_id=filename('Flickr8k_text/Flickr_8k.devImages.txt')#loadpath



In [ ]:
test_desc = load_desc("cleanedcaptions.txt",test_id)
#test_features = 

In [ ]:
test_desc

In [ ]:
test_features = load_features('features.pkl',test_id)

In [ ]:
#filepath = 'model-ep{epoch:03d}-loss{loss:.3f}-val_loss{val_loss:.3f}.h5'
#checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min')

In [ ]:
X1test, X2test, ytest = create_sequences(tokenizer, test_desc, test_features, maxlen )

In [ ]:
def model(vocab_size, max_length):
    
    input1_vgg = Input(shape = (4096, ))
    dropout_vgg = Dropout(0.5)(input1_vgg)
    dense_vgg = Dense(256, activation = 'relu')(dropout_vgg)
    
    input2_word = Input(shape = (max_length, ))
    embed_word = Embedding(vocab_size, 256, mask_zero = True)(input2_word)
    dropout_word = Dropout(0.5)(embed_word)
    lstm_word = LSTM(256)(dropout_word)
    
    adder = add([dense_vgg,lstm_word])
    dense = Dense(256, activation='relu')(adder)
    
    final = Dense(vocab_size, activation='softmax')(dense)
    
    model = Model(input=[input1_vgg,input2_word], output=[final])
    
    model.compile(loss='categorical_crossentropy', optimizer='adam')
    print(model.summary())
    plot_model(model, to_file='model.png', show_shapes=True)

    return model
    

In [ ]:
model = model(train_vocab_size, maxlen)

/Users/ajay/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:17: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=[<tf.Tenso..., outputs=[<tf.Tenso...)`


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 33)           0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, 4096)         0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 33, 256)      1860096     input_3[0][0]                    
__________________________________________________________________________________________________
dropout_1 (Dropout)             (None, 4096)         0           input_2[0][0]                    
__________________________________________________________________________________________________
dropout_2 

In [ ]:
#model.fit(([X1train,X2train]),ytrain,epochs=3,verbose=2,callbacks=[checkpoint], validation_data=([X1test,X2test],ytest))

In [ ]:
print('X1train.shape: ',X1train.shape,'\n')
print('X2train.shape: ',X2train.shape,'\n')
print('ytrain.shape: ',ytrain.shape,'\n')

print('X1test.shape: ',X1test.shape,'\n')
print('X2test.shape: ',X2test.shape,'\n')
print('ytest.shape: ',ytrain.shape,'\n')

X1train.shape:  (305617, 4096) 

X2train.shape:  (305617, 33) 

ytrain.shape:  (305617, 7266) 

X1test.shape:  (50835, 4096) 

X2test.shape:  (50835, 33) 

ytest.shape:  (305617, 7266) 



In [ ]:
def create_sequence(tokenizer,descs,train_feature,maxlen):
    for desc in descs:
        x,y,z = list(),list(),list()

        for desc in descs:
            seqofnumber = tokenizer.texts_to_sequences([desc])[0]
            for i in range(1,len(seqofnumber)):
                inseq = seqofnumber[:i]
                outseq = seqofnumber[i]
                inseq = pad_sequences([inseq],maxlen=maxlen)[0]

                outseq = to_categorical([outseq],num_classes=train_vocab_size)[0]
                y.append(inseq)
                z.append(outseq)
                x.append(train_feature)
    return np.array(x),np.array(y),np.array(z)

#model.fit is occupying a lot of RAM because the entire dataset is fed to the model for training
#model.fit_generator is used to feeding the dataset 

#In keras, fit() is much similar to sklearn's fit method, where you pass array of features as x values and target as y values.
#You pass your whole dataset at once in fit method. Also, use it if you can load whole data into your memory (small dataset).

#In fit_generator(), you don't pass the x and y directly, instead they come from a generator function. 
#This is for practical purpose, when you have large dataset. 

#so we create sequences for 1 photo at a time and then feed the sequence to model.fit_generator() train it to utilise less RAM.

#refer for generator function and yield https://stackoverflow.com/questions/231767/what-does-the-yield-keyword-do
#refer for fit and fit_generator https://datascience.stackexchange.com/questions/34444/what-is-the-difference-between-fit-and-fit-generator-in-keras



In [ ]:
#mygenerator = (x*x for x in range(3))
#type(mygenerator)

In [ ]:
def data_generator(tokenizer,train_descs,train_features,maxlen):
    while 1:
        for ids, descs in train_descs.items():
            feature = train_features[ids][0]
            feature_vector, inseq, outseq = create_sequence(tokenizer,descs,feature,maxlen)
            yield[[feature_vector,inseq],outseq]
generator = data_generator(tokenizer,train_descs,train_features,maxlen)

In [ ]:
inputs , outputs = next(data_generator(tokenizer,train_descs,train_features,maxlen))
#print(inputs)
#print(outputs)

In [ ]:
#print(inputs[0])
#print(inputs[1])

In [ ]:
#print(outputs)

In [ ]:
print(inputs[0].shape)
print(inputs[1].shape)
print(outputs.shape)

(47, 4096)
(47, 33)
(47, 7266)


In [ ]:
#
'''
epochs = 1
for i in range(epochs):
    generator = data_generator(tokenizer,train_descs,train_features,maxlen)
    # fit for one epoch
    model.fit_generator(generator,epochs=1,verbose=1,steps_per_epoch=133)
    model.save('model_'+i+'.h5')
'''

"\nepochs = 1\nfor i in range(epochs):\n    generator = data_generator(tokenizer,train_descs,train_features,maxlen)\n    # fit for one epoch\n    model.fit_generator(generator,epochs=1,verbose=1,steps_per_epoch=133)\n    model.save('model_'+i+'.h5')\n"

In [ ]:
len(train_descs)

6000

In [ ]:
#Refer to know about steps-per-epochs https://stackoverflow.com/questions/48604149/keras-fit-generator-and-steps-per-epoch
#Refer to know about tokenizer https://github.com/keras-team/keras/issues/7551#issuecomment-322105443
#

In [ ]:
def word_for_id(integer, tokenizer):
    for word,index in tokenizer.word_index.items():
        if index == integer:
            return word

In [ ]:
#word_for_id(4,tokenizer)

In [ ]:
#generates description for a given image by iteratively appending next predicted word to the sentence. 
def generate_desc(feature_vector,tokenizer,model,maxlen):
    sentence = 'startseq'
    for i in range(maxlen):
    
        seqofnumber = tokenizer.texts_to_sequences([sentence])[0]
        seqofnumber = pad_sequences([seqofnumber],maxlen=maxlen)

        predicted_vector = model.predict([feature_vector,seqofnumber],verbose=0)

        indexed_word = argmax(predicted_vector)
        word = word_for_id(indexed_wor,tokenizer)
        
        if word == None:
            break
        sentence = sentence + ' ' + word
        if word == 'endseq':
            break

    return sentence


In [ ]:
#generate_desc(,tokenizer,model,maxlen)
def p():
    print("imported successfully")

In [ ]:
s = ['sdf sd r','ioerm cscan vd','pqowa lsmx cvx']

r = [d.split() for d in s]
print(r)

[['sdf', 'sd', 'r'], ['ioerm', 'cscan', 'vd'], ['pqowa', 'lsmx', 'cvx']]


In [ ]:
pred_caption = "hello world this is"
predicted = list()
predicted.append(pred_caption.split())
predicted

[['hello', 'world', 'this', 'is']]

In [ ]:
#'description' in link mlmastery = 'mapping' in this code which are dictionaries of {id:[desc1,desc2,desc3]}
#check for bleu_score parameters https://stackoverflow.com/questions/40542523/nltk-corpus-level-bleu-vs-sentence-level-bleu-score
'''
see the no. associated with the below code lines
#actual codeblock
[#3
    [#1              #many sentences for 1 image
        [__,__,__],#2 #these 3 are different reference sentences for just 1 image(many sentences mapped with 1 imageid)
        [__,__,__],#2
        [__,__,__] #2
    ],
    
    [ #1            #many sentences for 2nd image
        [__,__,__],#2
        [__,__,__],#2
        [__,__,__] #2
    ]#refer
]#actual codeblock ends

#predicted codeblock
[#0
    [__,__,__],#1st predicted sentence #5
    [__,__,__],#2nd predicted sentence #5
    [__,__,__],#3rd predicted sentence #5
]

'''

def evaluate_model(mapping,tokenizer,maxlen,model,feature_vector):
    actual,predicted = list(),list()#0
    for ids,descs in mapping.items():#1
        pred_caption = generate_desc(feature_vector[ids],tokenizer,model,maxlen)#caption string returned 
        for desc in descs:#2
            reference.append(desc.split())
        actual.append(reference)#3
        predicted.append(pred_caption.split())#5
        
        #calculate BLEU scores for 1, 2, 3 and 4 cumulative n-grams.
        #A higher score close to 1.0 is better, a score closer to zero is worse.
    print('BLEU-1: %f' % corpus_bleu(actual, predicted, weights=(1.0, 0, 0, 0)))
    print('BLEU-2: %f' % corpus_bleu(actual, predicted, weights=(0.5, 0.5, 0, 0)))
    print('BLEU-3: %f' % corpus_bleu(actual, predicted, weights=(0.3, 0.3, 0.3, 0)))
    print('BLEU-4: %f' % corpus_bleu(actual, predicted, weights=(0.25, 0.25, 0.25, 0.25)))
#BLEU scores are used in text translation for evaluating translated text against one or more reference translations.

In [ ]:
evaluate_model(mapping,tokenizer,maxlen,model,features)

In [ ]:
features

In [ ]:
#split() method returns a list of strings after breaking the given string by the specified separator.
desc_list = ["hi can you run","DJ Bravo is here","his name is gb"] 
references = [d.split() for d in desc_list]
references

In [ ]:
#another alternative of doing the above task
alternative = list()
for desc in desc_list:
        alternative.append(desc.split())
print(alternative)

In [ ]:
type(train_features)

In [ ]:
print(type(mapping))
print(mapping)

In [ ]:
type(mapping['1000268201_693b08cb0e'])

In [ ]:
type(features['1000268201_693b08cb0e'][0])

In [ ]:
features['1000268201_693b08cb0e'].shape

In [ ]:
features['1000268201_693b08cb0e']

In [ ]:
type(features)

In [ ]:
#inside model.fit [X1train,X2train] X1train is numpy-array and X2train is numpy-array

In [ ]:
'''
When we call # evaluate model
evaluate_model(model, test_descriptions, test_features, tokenizer, max_length)

we know that test_features  which is a dictionary and contains id and a .
ie - { id1: [ [ 4096 elements] ], id2: [ [ 4096 elements] ] }.

Inside evaluate_model() we call -  generate_desc(model, tokenizer, photos[key], max_length)

where photos[key] is numpy.ndarray and hold the 4096 elements in another numpy.array.

To prove this I ran:
Note: feature is same as photo here, which contain id and its corresponding 4096 vector.

print(type(features['1000268201_693b08cb0e']))
print(type(features['1000268201_693b08cb0e'][0])
print(features['1000268201_693b08cb0e'].shape)
print(features['1000268201_693b08cb0e'])

and it gives me this output :

numpy.ndarray
numpy.ndarray
(1, 4096)
array([[ 2.5076463, -0.       , -0.       , ..., -0.       , -0.       ,
        -0.       ]], dtype=float32)


yhat = model.predict([photo,sequence], verbose=0) 

here photo[key] is a  np-array [ [ 4096 features inside ] ]. ie- array inside array.
'''

In [ ]:
'''
reality
photo
[
    [],
    [],
    [],
    []
]
'''

In [ ]:
'''
In generate_desc, photo is photo[key]. ie since photo = { id1: [ [ 4096 elements] ], id2: [ [ 4096 elements] ] }
                                            So, photo[key] = [ [ 4096 elements ] ]
so, photo = [ [ 4096 elements ] ]

so, during call to model.predict() inside generate_desc()
we pass [photo,sequence] and hence photo which is [ [ 4096 elements ] ] is correctly passed because during 
        model.fit_generator(generator, epochs=1, steps_per_epoch=steps, verbose=1)
        if we print generator we get [x,y] where a is [x1,x2]. 
        y is next word to predict and x1 is photo, x2 is sequence of past encountered word.
        To know what is x1,x2(ie-their type,shape). 
        we go into data_generator(), and we can see that it yields/returns [[in_img, in_seq], out_word]
        and see that in_img,in_seq,out_word are returned by create_sequences
        create_sequences() returns np.array(list()),np.array(list()),np.array(list())
        In first, each element is an array and the array contain 4096 elements  
        In second in_img, each element is an array(array of numbers each no. is token_no of a word encountered in previous steps) which can be seen by output  
        In third out_img, each element is a array((initially a number but now an array in which 1 is present at index x where x is number from string seq[i]).
 
                X1.append(train_features[key][0])#train_features is a dictionary and not a list.
                #train_features = {'id' : [], 'id': []}
                X2.append(inseq)
                y.append(outseq)
'''

In [ ]:
token_filehandler = open('tokenizer.pkl','wb')
dump(tokenizer,token_filehandler)
